In [187]:
#sorted([100, 200, 210, 300, 200, 210, 100, 200, 300, 200, 200, 100, 300, 190, 200, 100, 300, 190, 100, 300],reverse=True)

In [188]:
import gurobipy as gp
import numpy as np
import pandas as pd

Read data

In [189]:
import re

def parse_dzn(file_path):
    data = {}
    with open(file_path, 'r') as file:
        content = file.read()
        
        # Find all key-value pairs
        key_value_pairs = re.findall(r'(\w+)\s*=\s*(.*?);', content, re.DOTALL)
        
        for key, value in key_value_pairs:
            # Remove any extra whitespace and newlines
            value = value.strip().replace('\n', '')

            #Parse matrix
            if key == 'D':
                rows = value.split('|')
                D = []
                for i in range(1,len(rows)-1):
                    D.append([int(v.strip()) for v in rows[i][1:-1].split(',')])
                value = D
            # Parse arrays
            elif value.startswith('[') and value.endswith(']'):
                value = value[1:-1].split(',')
                value = [int(v.strip()) for v in value]
            else:
                value = int(value)
                
            data[key] = value
    return data

# Usage
'''
file_path = 'inst08.dzn'  # Path to your .dzn file
parsed_data = parse_dzn(file_path)
print(parsed_data)
'''

"\nfile_path = 'inst08.dzn'  # Path to your .dzn file\nparsed_data = parse_dzn(file_path)\nprint(parsed_data)\n"

Create the model

In [190]:
model = gp.Model()

Parse the instance

In [191]:
innum = input('Type instance number:\n>')
file_path = str(f'inst{innum}.dzn')
parsed_data = parse_dzn(file_path)

m = parsed_data['m']
n = parsed_data['n']
s = parsed_data['s']
l = parsed_data['l']
D = parsed_data['D']

maxD = np.max(D)

Define the variables

In [192]:
#distances = model.addVars(m,n+1,lb=0, vtype=gp.GRB.INTEGER, name='Distances')

max_total_dist = model.addVar(vtype=gp.GRB.CONTINUOUS, name='MaxTotalDist')
#max_total_dist_for_c = model.addVars(m, vtype=gp.GRB.INTEGER, name='MaxTotalDistForC')
#max_load_for_c = model.addVars(m, vtype=gp.GRB.INTEGER, name='MaxLoadForC')

cumulative_dist = model.addVars(m, n+1, vtype=gp.GRB.INTEGER, name='CumulativeDist')
cumulative_load = model.addVars(m, n+1, lb=0, ub=n*max(s), vtype=gp.GRB.INTEGER, name='CumulativeLoad')

visited = model.addVars(m,n,vtype=gp.GRB.BINARY, name='Visited')

paths = model.addVars(m, n+1, n+1, vtype=gp.GRB.BINARY, name='Paths')

Define Objective function

In [193]:
model.setObjective(max_total_dist, gp.GRB.MINIMIZE)

Define Constraints

In [194]:
#Maximum total distance is equal to the maximum total distance computed in the final depot "slot" for each courier
model.addConstrs( gp.quicksum(cumulative_dist[c,j] for j in range(n+1)) <= max_total_dist for c in range(m))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>}

In [195]:
#Every node must be visited exactly once (except for the depot n+1)
for dest in range(n):
    model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) <= 1)
    model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) >= 1)
    #model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) == 1)

In [196]:
for c in range(m):
    #Every courier must visit the depot as last node
    model.addConstr(gp.quicksum(paths[c,source,n] for source in range(n)) == 1)

    #Each courier must start from depot
    model.addConstr(gp.quicksum(paths[c,n,dest] for dest in range(n)) == 1)

    for source in range(n+1):
        #Couriers cannot stay still
        model.addConstr(paths[c,source,source] <= 0.5) #npe

        for dest in range(n+1):
            #Cumulative distance update
            model.addConstr((paths[c,source,dest] == 1) >> (cumulative_dist[c,dest] == D[source][dest])) #npe

            #Path contiguity
            model.addConstr((paths[c,source,dest] == 1) >> (gp.quicksum(paths[c,j,source] for j in range(n+1)) >= 1))
            model.addConstr((paths[c,source,dest] == 1) >> (gp.quicksum(paths[c,dest,j] for j in range(n+1)) >= 1))

            #Load update
            if dest < n:
                model.addConstr((paths[c,source,dest] == 1) >> (cumulative_load[c,dest] == s[dest])) #npe

        #Just in order to speed up the search    
        model.addConstr(gp.quicksum(paths[c,source,j] for j in range(n+1)) <= 1)

In [197]:
#Loop avoidance
for c in range(m):
    for source in range(n):
        for dest in range(source+1):
            #Just in order to speed up the search 
            #model.addConstr( (paths[c,source,dest] == 1) >> (paths[c,dest,source] <= 0.5) )

            #Visited update
            #model.addConstr( (paths[c,source,dest] == 1) >> (visited[c,source] == 1) )

            #Inner loops avoidance
            #model.addConstr( (visited[c,dest] == 1) >> ( paths[c,source,dest] == 0) )

            #Alternative
            model.addConstr( (gp.quicksum(paths[c,dest,j] for j in range(source+1)) + paths[c,source,dest]) <= 1)

In [198]:
#Symmetry breaking constraints
for c in range(m-1):
    model.addConstr( gp.quicksum(paths[c,source,dest] for source in range(n) for dest in range(n)) >= \
                     gp.quicksum(paths[c+1,source,dest] for source in range(n) for dest in range(n)) )

In [199]:
#Maximum load
for c in range(m):
    model.addConstr(gp.quicksum(cumulative_load[c,j] for j in range(n+1)) <= l[c])

In [201]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1265 rows, 2371 columns and 18302 nonzeros
Model fingerprint: 0x9694ca2b
Model has 7700 general constraints
Variable types: 1 continuous, 2370 integer (2090 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+02]
  RHS range        [5e-01, 2e+02]
  GenCon rhs range [1e+00, 2e+02]
  GenCon coe range [1e+00, 1e+00]
Presolve added 4985 rows and 1408 columns
Presolve time: 0.23s
Presolved: 6250 rows, 3779 columns, 54341 nonzeros
Presolved model has 1820 SOS constraint(s)
Variable types: 0 continuous, 3779 integer (1950 binary)

Root relaxation: objective 0.000000e+00, 615 iterations, 0.03 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

In [202]:
for c in range(m):
    print(c)
    rows = [i for i in range(n+1)]
    adj_matrix = pd.DataFrame(columns=rows, index=rows)
    for i in rows:
        for j in rows:
            adj_matrix.loc[i,j] = int(paths[c,i,j].X)
    print(adj_matrix)

0
   0  1  2  3  4  5  6  7  8  9  10 11 12 13
0   0  0  0  0  0  0  0  0  0  0  0  0  0  0
1   0  0  0  0  0  0  0  0  0  0  0  0  0  0
2   0  0  0  0  0  0  0  0  0  0  0  0  0  0
3   0  0  0  0  0  0  0  0  0  0  0  0  0  1
4   0  0  0  0  0  0  0  0  0  0  0  0  0  0
5   0  0  0  0  0  0  0  0  0  0  0  0  0  0
6   0  0  0  0  0  0  0  0  0  0  0  0  0  0
7   0  0  0  0  0  0  0  0  0  0  0  0  0  0
8   0  0  0  0  0  0  0  0  0  0  0  0  0  0
9   0  0  0  0  0  0  0  0  0  0  0  0  0  0
10  0  0  0  1  0  0  0  0  0  0  0  0  0  0
11  0  0  0  0  0  0  0  0  0  0  0  0  0  0
12  0  0  0  0  0  0  0  0  0  0  0  0  0  0
13  0  0  0  0  0  0  0  0  0  0  1  0  0  0
1
   0  1  2  3  4  5  6  7  8  9  10 11 12 13
0   0  0  0  0  0  0  0  0  0  0  0  0  0  0
1   0  0  0  0  0  0  0  1  0  0  0  0  0  0
2   0  0  0  0  0  0  0  0  0  0  0  0  0  0
3   0  0  0  0  0  0  0  0  0  0  0  0  0  0
4   0  0  0  0  0  0  0  0  0  0  0  0  0  0
5   0  0  0  0  0  0  0  0  0  0  0  0  0  0
6   0 